In [5]:
from pyspark.sql import *
import pandas as pd
from time import time

In [6]:
# Config
input_path='D:/Output/chenbingying/A2.csv'
output_path='D:/Output/chenbingying/Check/'

In [7]:
def Check_A2(input_path,output_path):
    t_start=time()
    spark = SparkSession.builder.appName('A2_Check').config('spark.dynamicAllocation.enabled','true').config('spark.debug.maxToStringFields', '100').config('spark.sql.execution.arrow.enable','true').config('spark.executor.memory','48g').config('spark.driver.memory', '48g').config('spark.core.connection.ack.wait.timeout','36000s').config('spark.executor.heartbeatInterval','36000s').config('spark.network.timeout', '50000s').config('spark.rpc.lookupTimeout', '5000s').config('spark.shuffle.io.connectionTimeout', '50000s').getOrCreate()
    print("正在处理：",input_path)
    start=time()
    spark.read.option('inferSchema','true').option('header','true').csv(input_path).createOrReplaceTempView('A2_Raw')
    print('读入数据耗时：%.2f秒'%(time()-start))
    start=time()
    df=spark.sql('''select 
        count(*) as TOTAL
        ,count(ID_NUMBER) as ID_NUMBER
        ,count(BIRTH_DATE) as BIRTH_DATE
        ,count(GENDER) as GENDER
        ,count(CITY) as CITY
        ,count(DISTRICT) as DISTRICT
        ,sum(case when MEDICAL_INSTITUTION_CODE is not null and FILE_NUMBER is not null then 1 else 0 end) as MF
        ,sum(case when CARD_TYPE is not null and CARD_NUMBER is not null then 1 else 0 end) as CC
        from A2_Raw
    ''').toPandas()
    for i in df.columns:
        if(i!='TOTAL'):
            df.insert(df.columns.get_loc(i)+1,i+'_RATE',df[i]/df['TOTAL']*100)
    print('处理数据耗时：%.2f秒'%(time()-start))
    df.columns=['数据总量','ID_NUMBER非空数据量','ID_NUMBER完整率','BIRTH_DATE非空数据量','BIRTH_DATE完整率','GENDER非空数据量','GENDER完整率','CITY非空数据量','CITY完整率','DISTRICT非空数据量','DISTRICT完整率','MEDICAL_INSTITUTION_CODE,FILE_NUMBER字段组合非空个数==2数据量','MEDICAL_INSTITUTION_CODE,FILE_NUMBER字段组合非空个数==2占比','CARD_TYPE,CARD_NUMBER字段组合非空个数==2数据量','CARD_TYPE,CARD_NUMBER字段组合非空个数==2占比']
    df.to_csv(output_path+'A2统计.csv',index=None)
    spark.catalog.dropTempView("A2_Raw")
    print(f'已输出至：{output_path}A2统计.csv')
    print('总耗时：%.2f秒'%(time()-t_start))

In [8]:
if __name__=="__main__":
    Check_A2(input_path,output_path)

正在处理  D:/Output/zhuhao/A2.csv
读入数据耗时：37.88秒
处理数据耗时：44.76秒
已输出至：D:/Output/zhuhao/Check/A2统计.csv
总耗时：82.65秒
